# 第6回講義　演習　SeqGAN

## 0. 導入

### 0.1. 強化学習

強化学習では、

1. 時刻tにおいて、状態(state):$s_t$にいるエージェント(agent)が行動(action):$a_t$をとる
1. エージェントは行動$a_t$によって状態$s_{t+1}$に遷移し、即時報酬(immediate reward):$r_{t+1}$を受け取る
1. 以上を繰り返す

という一連の流れでエージェントが得る割引報酬の和

$R=\sum_{t=0}^∞ \gamma^t r_{t+1}$

を最大化するようにエージェントを学習させることが目的です。

また終端状態(terminal state): $s_T$が存在する様なタスクはepisodicであるといい、初期状態から終端状態までをepisodeと呼びます。episodic taskの時のみ割引率$\gamma$は1にできます。

例えば囲碁のようなゲームはepisodicです。

この演習ではepisodic taskを扱うとし、割引率は$\gamma$は1、初期状態を$s_0$、最終時刻を$t=T$と定めます。

---

他の重要な構成要素

* **方策(Policy)**: ある状態sでのエージェントがとる行動aを決める関数
    * Deterministic policy: ある状態sでは必ず行動aをとる
        * $a = \pi (s)$
    * Stochastic policy: ある状態sで行動aをとる確率を定義 （← SeqGANはこっち）
        * $\pi (a|s)$
* **価値関数(Value function)**:
    * $V(s)$, ある状態s以降における(割引)報酬の和の期待値
* **行動価値関数(Action-value function)**
    * $Q(s, a)$, ある状態sで行動aを取ったあとの(割引)報酬の和の期待値

これらの定義より、
\begin{align}
V(s_{t})=\sum_{a_t} \pi (a_t|s_t) Q(s_t, a_t)\\
Q(s_t, a_t)=r(s, a)+V(s_{t+1})
\end{align}

と書ける。

強化学習の手法を大別

* Value based RL
    * Value function$V$またはAction-value function $Q$をパラメータで記述し、最適なパラメータを探す問題に帰着
    * 例：TD法、Q-learning, DQN, etc.
    * 利点：未知の状態$s$にも対応可能
* Policy based RL
    * Policy $\pi$をパラメータで記述し、以下同文
    * 例：Policy gradient
    * 利点：Stochastic policyも学習可能

まとめると、

|種別|パラメータで記述するもの|利点|アルゴリズム例|
|---|---|---|---|
|Value based RL|Value function $V$ または $Q$|未知の状態 s にも対応可能|TD法、Q-learning, DQN, etc.|
|Policy based RL|Policy $\pi$|Stochastic policyも学習可能|Policy gradient|

---

### 0.2. SeqGANにおける強化学習

<img src="./image/seqgan.png" width=600>
<div style="text-align: center;">
出典:[SeqGAN: Sequence Generative Adversarial Nets with Policy Gradient](https://arxiv.org/abs/1609.05473)
</div>

SeqGANとは列(sequence)の生成を行うためのGANです。教師あり学習だけではなく、強化学習の1手法であるPolicy gradientを用いて学習させます。

#### 0.2.1. なぜ強化学習を用いる必要があったのか

Teacher Forcingによる最尤推定(MLE)でも列生成モデルは学習することができます。しかしRNNをMLEで学習すると、訓練時はターゲット系列をそのまま入力として学習するがテスト時は自分の出力が次の時刻での入力となるために訓練時に見たことのない入力がテスト時に出てくる問題(_"exposure bias"_)が発生する可能性が高いです。この問題への対策の一つが訓練時にも一定の確率で自分の出力を次の時刻の入力とする手法がscheduled sampling (Bengio et al., 2015) でした。（第３回演習参照）しかしこれは本質的な問題の解決にはならないという報告もあります(Husz´ar, 2015)。

新たな解決策として、生成モデルの学習方法であるGAN (Generative Adversarial Network) の枠組みを文生成に適用し、さらに出力が離散値であるテキスト生成でも強化学習による学習を可能にしたのがこの論文の新規性です。

これによって、教師あり学習のみの場合よりも、よりTrue dataと似たような列を生成することができます。
例えば大量の詩のデータをTrue dataとしてSeqGANに学習させれば、それに似た新たな詩を生成することができます。データは列であれば良いので、音楽にも適用可能です。

#### 0.2.2. SeqGANにおける強化学習

SeqGANの実装における構成要素は以下のようになっています。

* True data
    * 実世界に存在するデータ。この演習では夏目漱石の『こころ』を扱います。


* Generator: G
    * LSTMからなるモデルです。Generated dataを出力します。True dataと似たデータをGeneratorが出力できるようにすることが目的です。


* Discriminator: D
    * CNNベースの2値分類モデルです。dataがTrue dataなのかGenerated dataなのかを見分けるためのモデルです。


* ROLLOUT: モンテカルロ探索 → 報酬計算
    * Generator Gを使って各時刻でモンテカルロ探索を行い、出力候補を多数生成（=rollout）します。そしてそれぞれの出力候補に対して、True dataである確率をDiscriminatorが計算し、それを報酬としてGeneratorに渡すことで強化学習を行います。

これらを用いて列（文）生成を強化学習の枠組みに当てはめると以下のようになります。

* 文には終端状態があるのでepisodicである
* 系列は$Y_{1:T}=(y_1,...,y_t,...,y_T),~y_t \in {\mathcal Y}$（${\mathcal Y}$ は語彙の集合）と表す
* 各時刻に1つの単語を生成する
* 初期状態$s_0$は何も単語を生成していない状態
* 状態$s_t$は単語列$Y_{1:t}$に相当
* 行動$a_t$は単語$y_{t+1}$を出力すること
* Generator $G_{\theta}(y_t|Y_{1:t-1})$
* 文がTrue dataである確率 $D_\phi (Y_{1:T})$をDiscriminatorが予測し、それを最終状態$s_T=Y_{1:T}$に対する報酬 $r_T$ とする。
* 報酬を計算するDiscriminatorの性質上、即時報酬は$r_t = 0~(t < T)$

文生成において、GeneratorをPolicy $\pi$ とみなしてパラメータ$\theta$で記述したいので、Policy gradientによって強化学習を行います。

### 0.3. Policy Gradient （方策勾配法）とは

一言で言うとPolicyをパラメータで記述し、勾配法によって最適なPolicyを探すことを指します。

ある状態$s_t=Y_{1:t-1}$でどの行動$a_t=y_t$をとるか(方策, policy)を決める方策関数(policy function)、$\pi(a_t|s_t;\theta)=G(y_t|Y_{1:t-1})$を考えます。$\theta$はパラメータです。

この方策関数$\pi$に従った時の最終的な報酬の和の期待値$V(s_0)$$(=J(\theta))$を勾配法で最大化する手法が方策勾配法(policy gradient)です。

以下のように$\theta$を更新していくことで方策関数を更新して行き、期待報酬を最大化します。

$\theta \gets \theta + \alpha \nabla J(\theta)$

$\alpha$は定数（$\alpha > 0$）です。

$\nabla J(\theta)$は以下のように表せます。
まずPolicy Gradient Theorem (Sutton et al., 2000) により、

\begin{align*}
& \nabla J(\theta) = \nabla V(s_0) \\
& = \nabla \left[ \sum_{y_1 \in {\mathcal Y}} G_\theta (y_1|s_0) Q(s_0, y_1) \right] \\
& = \sum_{t=1}^T \sum_{Y_{1:t-1}} P(Y_{1:t-1}|s_0;G_\theta) \sum_{y_t \in {\mathcal Y}} \nabla G_\theta (y_t|Y_{1:t-1}) Q(Y_{1:t-1}, y_t) \\
\end{align*}

$\nabla f = f \nabla \log f$より、

\begin{align*}
& = \sum_{t=1}^T \sum_{Y_{1:t-1}} P(Y_{1:t-1}|s_0;G_\theta) \sum_{y_t \in {\mathcal Y}} G_\theta (y_t|Y_{1:t-1}) \nabla \log G_\theta (y_t|Y_{1:t-1}) Q(Y_{1:t-1}, y_t) \\
& = \sum_{t=1}^T \sum_{Y_{1:t}} P(Y_{1:t}|s_0;G_\theta)  \nabla \log G_\theta (y_t|Y_{1:t-1}) Q(Y_{1:t-1}, y_t) \\
& = \sum_{t=1}^T {\mathbb E}_{Y_{1:t} \sim G_\theta} \left[ \nabla \log G_\theta (y_t|Y_{1:t-1}) Q(Y_{1:t-1}, y_t) \right]
\end{align*}

※詳しい証明は[論文](https://arxiv.org/pdf/1609.05473.pdf)のAppendix参照

### 0.4. Monte Carlo Policy Gradient

0.3.のように、$\nabla J(\theta)$を計算できれば良いのですが、2つ問題があります。1つ目は${\mathbb E}_{Y_{1:t} \sim G_\theta} \left[ \cdot \right]$をどう計算するか、2つ目はどうやって$Q(Y_{1:t-1}, y_t)$を計算するかです。

1つ目の${\mathbb E}_{Y_{1:t} \sim G_\theta} \left[ \cdot \right]$はPolicy $G_\theta$を使って$Y_{1:t}$をサンプリングすることで推定できます。

2つ目の$Q(Y_{1:t-1}, y_t)$は、Monte Carlo法で推定します。Discriminatorは文全体$Y_{1:T}$に対してしか報酬を与えることができないので、$Y_{t+1:T}$をRollout policy $G_\beta$を使ってランダムにN回サンプリングして得られた結果$\{ Y^1_{1:T},...,Y^N_{1:T}\} = MC^{G_\beta}(Y_{1:t};N)$への報酬の平均を$Q(Y_{1:t-1}, y_t)$とします。こうすることで中間の状態に対しても$Q$を定義できます。

\begin{align}
&Q(Y_{1:t-1}, y_t)=\\
&\begin{cases}
\frac{1}{N}\sum_{n=1}^ND_\phi(Y^n_{1:T}),~Y^n_{1:T} \in MC^{G_\beta}(Y_{1:t};N) & {\rm for}~ t < T\\
D_\phi(Y_{1:t}) & {\rm for}~t=T
\end{cases}
\end{align}

### 0.4. 学習の流れ

学習は以下の流れで行います。
* Pretraining（事前学習）
  1. GeneratorをTrue dataで事前学習（**教師あり学習**）
  1. Discriminatorを教師あり学習

を行なった後に

* Adversarial Training（敵対的学習）
  1. GeneratorをPolicy Gradientで学習（**強化学習**）
  1. Discriminatorを教師あり学習
  1. 1に戻る

In [1]:
import os
import copy
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

try:
    from utils import Vocab
except ModuleNotFoundError:
    os.chdir('/root/userspace/chap6/')
    from utils import Vocab
    
torch.manual_seed(1)

## 1. ハイパーパラメータ設定

In [3]:
# Generator Hyper-parameters
G_EMBEDDING_SIZE = 32 # 埋め込みベクトルの次元数
G_HIDDEN_SIZE = 32 # LSTMの隠れ状態ベクトルの次元数
G_MAX_LENGTH = 40 # 系列の長さ
# G_PRE_NUM_EPOCHS = 120 # 事前学習を行うエポック数
G_PRE_NUM_EPOCHS = 1
G_BATCH_SIZE = 64 # バッチサイズ

# Discriminator Hyper-parameters
D_EMBEDDING_SIZE = 64
D_FILTER_SIZES = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20] # CNNに使うフィルターのサイズ
D_NUM_FILTERS = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160] # CNNに使うフィルターの数
D_DROPOUT_PROB = 0.75 # Dropoutの確率
# D_PRE_NUM_EPOCHS = 50
D_PRE_NUM_EPOCHS = 1
D_BATCH_SIZE = 64

# Basic Training Parameters
# ADV_N_BATCHES = 200 # 敵対的学習を行うエポック数
ADV_N_BATCHES = 1
POSITIVE_FILE = './save/real_data.txt'
NEGATIVE_FILE = './save/generator_sample.txt'
# GENERATED_NUM = 10000
GENERATED_NUM = 1000 # 敵対的学習のためにGeneratorに出力させるサンプル数

# 保存用ディレクトリがなければ作成する
if not os.path.exists('./save'):
    os.mkdir('./save')

## 2. データ
夏目漱石の『こころ』を学習データに使います。

### 2.1. データ読み込み・前処理

In [4]:
# 特殊なトークンを事前に定義します
PAD_TOKEN = '<PAD>'
BOS_TOKEN = '<S>'
EOS_TOKEN = '</S>'
UNK_TOKEN = '<UNK>'
PAD = 0
BOS = 1
EOS = 2
UNK = 3

In [5]:
def load_data(path):
    """スペース区切りのコーパスを1行ごとに読み込む関数

    :path: str, ファイルのパス
    :return text: list of list of str
    """
    text = []
    with open(path, "r") as f:
        for line in f:
            text.append(line.strip().split())
    return text

In [6]:
# 事前にこちらで分かち書きしたものを用意したのでこれを使ってください。
text = load_data("./data/kokoro_parsed.txt")
print(text[0])

['私', 'は', 'その', '人', 'を', '常に', '先生', 'と', '呼ん', 'で', 'い', 'た', '。']


In [7]:
MIN_COUNT = 1 # 語彙に含める単語の最低出現回数

word2id = {
    PAD_TOKEN: PAD,
    BOS_TOKEN: BOS,
    EOS_TOKEN: EOS,
    UNK_TOKEN: UNK,
}

vocab = Vocab(word2id=word2id)
vocab.build_vocab(text, min_count=MIN_COUNT)
print("語彙数\t:", len(vocab.word2id))

語彙数	: 6661


In [8]:
# 統計量
lens = [len(x) for x in text]
print("文の総数\t:", len(text))
print("文長の最大値\t:", np.max(lens))
print("文長の平均\t:", np.mean(lens))
print("標準偏差\t:", np.std(lens))

文の総数	: 4267
文長の最大値	: 283
文長の平均	: 24.62385751113194
標準偏差	: 17.948284504112998


In [9]:
def sentence_to_ids(vocab, sentence):
    """単語のリストを単語IDのリストに変換する関数

    :param vocab: Vocabのインスタンス
    :param sentence: list of str
    :return ids: list of int
    """
    ids = [vocab.word2id.get(word, UNK) for word in sentence]
    ids = [BOS] + ids + [EOS]  # </S>トークンを末尾に加える
    return ids

In [10]:
def pad_seq(sen, max_length):
    """padding, truncatingを行う

    :param sen: list of int, 単語IDのリスト
    :param max_length: int, paddingを行なった後のsequenceの長さ
    :return sen: list of int, padding後のsequence
    """
    if len(sen) <= max_length:
        # Padding
        sen += [PAD] * (max_length - len(sen))
    else:
        # Truncating
        sen = sen[:max_length]
    return sen

In [11]:
# 単語をIDに変換して、paddingを行います

id_text = []
for sen in text:
    id_sen = sentence_to_ids(vocab, sen)
    id_sen = pad_seq(id_sen, G_MAX_LENGTH)
    id_text.append(id_sen)

In [12]:
# IDに変換したデータをファイルに保存します
# これを使ってGenerator、Discriminatorを学習させます

with open(POSITIVE_FILE, "w") as f:
    for id_sen in id_text:
        f.write(' '.join([str(w) for w in id_sen]) + '\n')

### 2.2. データローダー

Generator, Descriminatorそれぞれの学習のために個別にDataLoaderを用意します。
ともにデータはtxtファイルから読みます。

In [27]:
class GenDataLoader(object):
    """Generatorのためのデータローダー"""
    def __init__(self, file_path, batch_size):
        """
        :param file_path: str, 学習用データファイルのパス
        :param batch_size: int, ミニバッチのサイズ
        """
        super(GenDataLoader, self).__init__()
        self.data = self.load_file(file_path) # ファイル読み込み
        self.batch_size = batch_size # バッチサイズ
        self.pointer = 0 # ポインター
        self.data_num = len(self.data) # データの総数

        # データの順番をランダムに並び替える
        self.reset()

    def __iter__(self):
        return self

    def __next__(self):
        if self.pointer >= self.data_num:
            self.reset()
            raise StopIteration
        batch = torch.tensor(self.data[self.pointer:self.pointer + self.batch_size], dtype=torch.long)
        batch_X = batch[:, :-1].to(device) # 入力: <BOS>から<EOS>の手前まで
        batch_Y = batch[:, 1:].to(device) # 出力: <BOS>の次から<EOS>まで
        self.pointer += self.batch_size
        return batch_X, batch_Y

    def load_file(self, file_path):
        """データを読み込むメソッド

        :param file_path: str
        :return data: list of list of int
        """
        data = []
        with open(file_path, "r") as f:
            for line in f:
                line = line.strip().split()
                line = [int(x) for x in line]
                data.append(line)
        return data

    def reset(self):
        self.pointer = 0
        random.shuffle(self.data)

In [13]:
class DisDataLoader(object):
    """Discriminatorのためのデータローダー"""
    def __init__(self, positive_file, negative_file, batch_size):
        """
        :param positive_file: str, True Dataのパス
        :param negative_file: str, Generated Dataのパス
        :param batch_size: int, ミニバッチのサイズ
        """
        super(DisDataLoader, self).__init__()
        self.batch_size = batch_size
        pos_data = self.load_file(positive_file) # True data
        neg_data = self.load_file(negative_file) # Generated data
        self.data = pos_data + neg_data # 入力

        # 入力がTrue dataなら出力は1
        # 入力がGenerated dataなら出力は0
        self.labels = [1 for _ in range(len(pos_data))] + [0 for _ in range(len(neg_data))]
        self.pairs = list(zip(self.data, self.labels))
        self.data_num = len(self.pairs)
        self.pointer = 0

        self.reset()

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.pointer >= self.data_num:
            self.reset()
            raise StopIteration
        batch_X, batch_Y = zip(*self.pairs[self.pointer:self.pointer + self.batch_size])
        batch_X = torch.tensor(batch_X, dtype=torch.long, device=device) # 入力: True or Generated
        batch_Y = torch.tensor(batch_Y, dtype=torch.long, device=device) # 出力: 1 or 0
        self.pointer += self.batch_size
        return batch_X, batch_Y

    def load_file(self, file_path):
        """データを読み込むメソッド

        :param file_path: str
        :return data: list of list of int
        """
        data = []
        with open(file_path, "r") as f:
            for line in f:
                line = line.strip().split()
                line = [int(x) for x in line]
                data.append(line)
        return data

    def reset(self):
        self.pointer = 0
        random.shuffle(self.pairs)

## 3. モデル定義

### 3.1. Generator
一般的なLSTMを使います。

<img src="./image/generator.png" width=720>


In [14]:
class Generator(nn.Module):
    def __init__(self, vocab_size, batch_size, embedding_size, hidden_size,
                 max_length):
        """
        :param vocab_size: int, 語彙の総数
        :param batch_size: int, ミニバッチのサイズ
        :param embedding_size: int, 埋め込みベクトルの次元数
        :param hidden_size: int, 隠れ状態ベクトルの次元数
        :param max_length: int, 入出力系列の長さ
        """
        super(Generator, self).__init__()
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.max_length = max_length
        
        # 埋め込み層
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        # LSTM
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, 1, batch_first=True)
        # 全結合層
        self.linear = nn.Linear(self.hidden_size, self.vocab_size)

    def forward(self, seqs):
        """ターゲット系列を全時刻での入力として出力を計算

        Negative Log Likelihoodを計算するために使うので、softmaxではなくlog_softmaxを使います。

        :param seqs: torch.Tensor, (batch_size, max_length)
        :return outputs: torch.Tensor, (batch_size * max_length, vocab_size)
        """
        N = seqs.size(0) # batch_size

        # WRITE ME!
        embed = self.embedding(seqs) # (batch_size, max_length, embedding_size)

        h, c = self.init_hidden(N) # 隠れ状態ベクトルの初期化
        self.lstm.flatten_parameters()

        # WRITE ME!
        hidden, (h, c) = self.lstm(embed, (h, c)) # hidden:(batch_size, max_length, hidden_size)
        lin = self.linear(hidden)  # (batch_size, max_length, vocab_size)

        outputs = F.log_softmax(lin, dim=-1) # (batch_size, max_length, vocab_size)
        outputs = outputs.view(-1, self.vocab_size) # (batch_size * max_length, vocab_size)
        return outputs

    def step(self, x, h, c):
        """時刻をtからt+1に1つだけ進めます

        :param x: torch.Tensor, 時刻tの出力かつ時刻t+1の入力
        :param h, c: torch.Tensor, 時刻tの隠れ状態ベクトル
        :return pred: torch.Tensor, 時刻t+1の出力
        :return h, c: torch.Tensor, 時刻t+1の隠れ状態ベクトル
        """
        embed = self.embedding(x) # embed:(batch_size, 1, embedding_size)
        self.lstm.flatten_parameters()
        hidden, (h, c) = self.lstm(embed, (h, c)) # y:(batch_size, 1, hidden_size)
        pred = F.softmax(self.linear(hidden), dim=-1) # (batch_size, 1, vocab_size)
        return pred, h, c

    def sample(self, x=None):
        """Generaterでサンプリングするメソッド
        x == None (, flag is True)
            -> 全系列をサンプリング
        x == torch.Tensor, (batch_size, seq_length) (, flag is False)
            -> x以降の系列をサンプリング

        :param x: None or torch.Tensor, (batch_size, seq_length)
        :return output: torch.Tensor, (batch_size, max_length)
        """
        flag = False # 時刻0から始める(True)か否か(False)
        if x is None:
            flag = True
        if flag:
            x = torch.empty(self.batch_size, 1).fill_(BOS).long().to(device)
        h, c = self.init_hidden(self.batch_size)

        samples = []
        if flag:
            for i in range(self.max_length):
                output, h, c = self.step(x, h, c) # output:(batch_size, 1, vocab_size)
                output = output.squeeze(1) # (batch_size, vocab_size)
                x = output.multinomial(1) # (batch_size, 1), 次の時刻の入力をカテゴリカル分布からサンプリング
                samples.append(x)
        else:
            given_len = x.size(1)
            lis = x.chunk(x.size(1), dim=1) # max_length方向に分割
            for i in range(given_len): # xを出力し終わった時の隠れ状態ベクトルを再現する
                output, h, c = self.step(lis[i], h, c)
                samples.append(lis[i])
            output = output.squeeze(1)
            x = output.multinomial(1)
            for i in range(given_len, self.max_length): # モンテカルロ法
                samples.append(x)
                output, h, c = self.step(x, h, c)
                output = output.squeeze(1)
                x = output.multinomial(1) # 単語をサンプリング
        output = torch.cat(samples, dim=1) # (batch_size, max_length)
        return output

    def init_hidden(self, N):
        """LSTMの隠れ状態ベクトルを初期化します。

        :param N: int, ミニバッチのサイズ
        :return h0, c0: torch.Tensor, 初期状態での隠れ状態ベクトル
        """
        h0 = torch.zeros(1, N, self.hidden_size).to(device)
        c0 = torch.zeros(1, N, self.hidden_size).to(device)
        return h0, c0

### 3.3. Discriminator
与えられたsequenceがGenerated data(Generatorが生成したもの)かTrue dataかを判別するための2値分類器です。
モデルはCNNがベースになっていて、内部でHighway Networkを使います。

Highway NetworkとはNeural Network内の情報の流れをgateによって制限することで層が多いDeep Neural Networkでも学習を可能にすることができる機構です。

<img src="./image/discriminator.png" width=720>


In [15]:
class Highway(nn.Module):
    """Highway Network (cf. http://arxiv.org/abs/1505.00387)

    gate = sigmoid(Wx + b)
    nonlinear = f(W'x + b')
    y = gate * nonlinear + (1 - gate) * x

    ここで f はreluなどの非線形な活性化関数です。gateはtransform gate、(1 - gate)はcarry gateと呼ばれます。
    xからyを計算する過程がHighway Networkの1層に相当します。
    """
    def __init__(self, input_size, num_layers=1, f=F.relu):
        """
        :param input_size: int, 入力のサイズ
        :param num_layers: int, Highway Networkの層数
        :param f: 非線形な活性化関数
        """
        super(Highway, self).__init__()
        self.num_layers = num_layers
        self.linear1 = nn.ModuleList([nn.Linear(input_size, input_size) for _ in range(num_layers)])
        self.linear2 = nn.ModuleList([nn.Linear(input_size, input_size) for _ in range(num_layers)])
        self.f = f

    def forward(self, x):
        """
        :param x: torch.Tensor, (batch_size, input_size)
        :return x: torch.Tensor, (batch_size, input_size)
        """
        for layer in range(self.num_layers):
            gate = torch.sigmoid(self.linear1[layer](x))
            nonlinear = self.f(self.linear2[layer](x))
            x = gate * nonlinear + (1 - gate) * x
        return x

In [16]:
class Discriminator(nn.Module):
    def __init__(self, vocab_size, batch_size, embedding_size,
                num_filters, filter_sizes, dropout_prob=0.75):
        """
        :param vocab_size: int, 語彙の総数
        :param batch_size: int, ミニバッチのサイズ
        :param embedding_size: int, 埋め込みベクトルの次元数
        :param num_filters: list of int, CNNのフィルターの数のリスト
        :param filter_sizes: list of int, CNNのフィルターのサイズのリスト
        :param dropout_prob: float, ドロップアウトの確率
        """
        super(Discriminator, self).__init__()
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        assert len(self.num_filters) == len(self.filter_sizes)
        
        # 埋め込み層
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        # CNN
        self.convs = nn.ModuleList([nn.Conv2d(1, Co, (K, self.embedding_size))
                                    for Co, K in zip(self.num_filters, self.filter_sizes)])
        # Highway
        self.highway = Highway(sum(self.num_filters))
        # Dropout
        self.dropout = nn.Dropout(dropout_prob)
        # 全結合層
        self.linear = nn.Linear(sum(self.num_filters), 2)

    def forward(self, x, log=True):
        """
        :param x: torch.Tensor, (batch_size, max_length)
        :param log: bool, log_softmaxを使うかsoftmaxを使うかを決める
        :return x: torch.Tensor, (batch_size, 2)
        """
        x = self.embedding(x)  # (batch_size, max_length, embedding_size)
        x = x.unsqueeze(1)  # (batch_size, 1, max_length, embedding_size)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]  # [(batch_size, Co, Lo), ...]
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(batch_size, Co), ...]
        x = torch.cat(x, 1) # (batch_size, sum(self.num_filters))
        x = self.highway(x) # (batch_size, sum(self.num_filters))
        x = self.dropout(x) # (batch_size, sum(self.num_filters))
        x = self.linear(x) # (batch_size, 2)
        if log:
            x = F.log_softmax(x, dim=-1) # (batch_size, 2)
        else:
            x = F.softmax(x, dim=-1) # (batch_size, 2) # 真であるか確率、義である確率
        return x

## 4. Monte Carlo search→Rewardの計算

Monte Carlo (MC) searchとは複数回ランダムに探索を行うことです。囲碁や将棋などで次にどの手を打つのが最も好ましいか考えるときに、数手先まで考えて複数の候補を比較して次に打つべき手を考えるのと同じことです。

Rollout policy (ここではGenerator) にMC探索を行わせて得られた複数の系列に対してDiscriminatorが確率を計算し、報酬とします。

In [20]:
class ROLLOUT():
    def __init__(self, model, update_rate):
        """
        :param model: Generatorのインスタンス
        :param update_rate: モデルのパラメータの更新率
        """
        self.ori_model = model # Policy: G_\theta
        self.own_model = copy.deepcopy(model) # Rollout policy: G_\beta
        self.update_rate = update_rate # \alpha

    def get_reward(self, x, num, discriminator):
        """モンテカルロ探索とdiscriminatorを使ってGenerated dataの各時刻に対して報酬（True dataである確率）を与えます。

        :param x : torch.Tensor, Generated data, (batch_size, seq_length)
        :param num : モンテカルロ法でサンプリングをする回数
        :param discriminator : Discrimanatorのインスタンス
        :return rewards: torch.Tensor, 報酬（True dataである確率）, (batch_size, seq_length)
        """
        rewards = []
        batch_size = x.size(0)
        seq_length = x.size(1)
        for i in range(num):
            for t in range(1, seq_length):
                # Y_{1:t}に対して与える報酬を推定
                data = x[:, 0:t]
                samples = self.own_model.sample(x=data) # Y_{t:T}をモンテカルロ法でサンプリング, (batch_size, seq_length)
                pred = discriminator(samples, log=False) # True dataである確率を計算, (batch_size, 2)
                pred = pred.cpu().data[:,1].numpy() # (batch_size,)
                if i == 0:
                    rewards.append(pred)
                else:
                    rewards[t-1] += pred

            # Y_{1:T}に対して与える報酬を推定
            pred = discriminator(x, log=False)
            pred = pred.cpu().data[:, 1].numpy()
            if i == 0:
                rewards.append(pred)
            else:
                rewards[seq_length-1] += pred
        rewards = np.array(rewards) # (seq_length, batch_size)
        rewards = np.transpose(rewards) / (1.0 * num) # (batch_size, seq_length)
        return rewards

    def update_params(self):
        dic = {}
        for name, param in self.own_model.named_parameters():
            dic[name] = param.data
        for name, param in self.ori_model.named_parameters():
            if name.startswith('emb'):
                param.data = dic[name]
            else:
                # パラメータを更新した後と更新する前の内分点をとる(0.3節参照)
                param.data = (1 - self.update_rate) * param.data + self.update_rate * dic[name]
        self.own_model = copy.deepcopy(self.ori_model)

## 5. 関数定義

### 5.1. サンプル生成

In [21]:
def generate_samples(model, batch_size, generated_num, output_file):
    """generatorを使ってサンプル生成を行う関数
    
    :param model: モデル
    :param batch_size: int, バッチサイズ
    :param generated_num: int, 生成するサンプルの総数
    :param output_file: str, サンプルの出力先ファイル
    """
    samples = []
    for _ in range(int(generated_num / batch_size)):
        sample = model.sample().cpu().data.numpy().tolist() # (batch_size, max_length)
        samples.extend(sample)
    with open(output_file, 'w') as fout:
        for sample in samples:
            string = ' '.join([str(s) for s in sample]) + "\n"
            fout.write(string)

### 5.2. 訓練

In [22]:
def compute_loss(model, data_loader, criterion, optimizer=None, is_train=True):
    """GeneratorまたはDiscriminatorでlossを計算するための関数

    :param model: モデル
    :param data_loader: モデルのためのデータローダー
    :param criterion: nn.NLLLoss()などのlossを計算するクラスのインスタンス
    :param optimizer: Adamなどのoptimizer
    :param is_train: bool, trainするか否かを決める
    :return loss: float
    """
    model.train(is_train)

    total_loss = 0.
    total_batches = 0.

    for batch_X, batch_Y in data_loader:
        pred_Y = model(batch_X)
        loss = criterion(pred_Y, batch_Y.contiguous().view(-1))
        total_loss += loss.item()
        total_batches += 1

        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    data_loader.reset()
    loss = total_loss / total_batches
    return loss

## 6. 実験

学習は以下の流れで行います。
* Pretraining（事前学習）
  1. GeneratorをTrue dataで事前学習（**教師あり学習**）
  1. Discriminatorを教師あり学習

を行なった後に

* Adversarial Training（敵対的学習）
  1. GeneratorをPolicy Gradientで学習（**強化学習**）
  1. Discriminatorを教師あり学習
  1. 1に戻る

In [24]:
vocab_size = len(vocab.word2id)
generator = Generator(vocab_size, G_BATCH_SIZE, G_EMBEDDING_SIZE, G_HIDDEN_SIZE, G_MAX_LENGTH).to(device)
discriminator = Discriminator(vocab_size, D_BATCH_SIZE, D_EMBEDDING_SIZE,
                D_NUM_FILTERS, D_FILTER_SIZES, dropout_prob=D_DROPOUT_PROB).to(device)

### 6.1. Pre-train

In [28]:
# pretrain generator
gen_data_loader = GenDataLoader(POSITIVE_FILE, G_BATCH_SIZE)
gen_criterion = nn.NLLLoss()
gen_optimizer = optim.Adam(generator.parameters())
for epoch_id in range(1, G_PRE_NUM_EPOCHS + 1):
    gen_data_loader.reset()
    train_loss = compute_loss(
        generator, gen_data_loader, gen_criterion, optimizer=gen_optimizer, is_train=True)
    print("Epoch:{}, GenLoss:{:.4f}".format(epoch_id, train_loss))

Epoch:1, GenLoss:7.5381


In [29]:
# pretrain discriminator
dis_criterion = nn.NLLLoss()
dis_optimizer = optim.Adam(discriminator.parameters())
for epoch_id in range(1, D_PRE_NUM_EPOCHS + 1):
    generate_samples(generator, D_BATCH_SIZE, GENERATED_NUM, NEGATIVE_FILE)
    dis_data_loader = DisDataLoader(POSITIVE_FILE, NEGATIVE_FILE, D_BATCH_SIZE)
    for _ in range(3):
        loss = compute_loss(
            discriminator, dis_data_loader, dis_criterion, optimizer=dis_optimizer, is_train=True)
        print("Epoch:{}, DisLoss: {:.4f}".format(epoch_id, loss))
        break

Epoch:1, DisLoss: 0.3369


### 6.2. Adversarial training

Generator Gのパラメータ$\theta$の更新方法

\begin{align*}
& \theta \gets \theta + \alpha \nabla J(\theta) \\
\\
& \nabla J(\theta) = \frac{1}{T} \sum_{t=1}^T {\mathbb E}_{Y_{1:t} \sim G_\theta} \left[ \nabla \log G_\theta (y_t|Y_{1:t-1}) Q(Y_{1:t-1}, y_t) \right]\\
\\
&Q(Y_{1:t-1}, y_t)=\\
&\begin{cases}
\frac{1}{N}\sum_{n=1}^ND_\phi(Y^n_{1:T}),~Y^n_{1:T} \in MC^{G_\beta}(Y_{1:t};N) & {\rm for}~ t < T\\
D_\phi(Y_{1:t}) & {\rm for}~t=T
\end{cases}
\\
\end{align*}

${\mathbb E}_{Y_{1:t} \sim G_\theta} \left[ \cdot \right]$は$G_\theta$を使って$Y_{1:t}$をサンプリングすることで推定

In [31]:
# adversarial training
rollout = ROLLOUT(generator, 0.8)

gen_gan_criterion = nn.NLLLoss(reduce=False)
gen_gan_optimizer = optim.Adam(generator.parameters())
dis_criterion = nn.NLLLoss()
dis_optimizer = optim.Adam(discriminator.parameters())
for batch_id in range(1, ADV_N_BATCHES + 1):
    ## Train the generator for one step
    for it in range(1):
        samples = generator.sample() # (batch_size, max_length)
        # generatorへの入力を用意する＝BOSで始まるsequenceを用意する
        start_tokens = torch.empty(G_BATCH_SIZE, 1).fill_(BOS).type(torch.long)
        start_tokens = start_tokens.to(device) # (batch_size, 1)
        inputs = torch.cat([start_tokens, samples],
                           dim = 1)[:, :-1].contiguous() # (batch_size, max_length)
        targets = samples.contiguous().view((-1,)) # (batch_size * max_length,)
        # discriminatorで報酬を計算
        rewards = rollout.get_reward(samples, 16, discriminator) # (batch_size, max_length)
        rewards = torch.Tensor(rewards).contiguous().view((-1,)).to(device) # (batch_size * max_length,)

        # WRITE ME!
        log_prob = generator.forward(inputs) # log G_\theta(), (batch_size * max_length, vocab_size)
        train_loss = gen_gan_criterion(log_prob, targets)

        # WRITE ME!
        train_loss = (log_prob * rewards).mean() # train_lossにrewardをかけてから平均

        gen_gan_optimizer.zero_grad()
        train_loss.backward()
        gen_gan_optimizer.step()

    print('Batch: {}, TrainLoss: {:.4f}'.format(batch_id, train_loss.item()))

    rollout.update_params()

    for _ in range(4):
        generate_samples(generator, G_BATCH_SIZE, GENERATED_NUM, NEGATIVE_FILE)
        dis_data_loader = DisDataLoader(POSITIVE_FILE, NEGATIVE_FILE, D_BATCH_SIZE)
        for _ in range(2):
            loss = compute_loss(
                discriminator, dis_data_loader, dis_criterion, optimizer=dis_optimizer, is_train=True)
            break

RuntimeError: The size of tensor a (6661) must match the size of tensor b (2560) at non-singleton dimension 1

## 出力例

こちらで事前に学習した時のGeneratorのパラメータと辞書を用意したので、それらを読み込んで実際に文章を生成してみましょう。

In [32]:
# 学習済みパラメータを読み込む
from utils import TextGenerator
g = TextGenerator(vocab_size, G_BATCH_SIZE, G_EMBEDDING_SIZE, G_HIDDEN_SIZE, G_MAX_LENGTH).to(device)
g.load_state_dict(torch.load("./data/trained_generator_params.pth"))
g.eval()
samples = g.sample()

In [33]:
# 学習時と同じ辞書を読み込む
import pickle
with open("./data/id2word.pickle", "rb") as f:
    id2word = pickle.load(f)

In [34]:
for s in samples.data.cpu().numpy():
    print(" ".join(filter(lambda x: x != "<PAD>", [id2word[w] for w in s])))

私 の 問題 に いう という 自由 で あっ た 。 </S>
私 の 駄菓子 盤 を 構成 し た の 背後 で 実現 する って ある 。 </S>
私 の 前 に しばしば あっ た 。 </S>
私 の です 。 </S>
私 の 中 で あっ た 。 </S>
帰っ て 黙っ て 下さい 。 </S>
私 の 話 を 制する れ です 。 </S>
お上 として 、 彼 の 頭 の 意外 に 歌留多 を 拭い た 。 </S>
私 の 勉強 、 来 た 。 </S>
夕飯 の 親類 の 悪い から も 、 私 の 家 を 開く 事 に 使い まし た 。 </S>
私 の 中 が 常に 父 の ため に つい た 。 </S>
私 が 乗ら ない 熊 の 哲理 で あっ た 。 </S>
拾い読み に なっ た 。 </S>
私 の 室 から 出 た 。 </S>
私 に 同時に 私 から 、 凝 の 顔 の ある もの で も 私 が 、 湯呑 を 解き ろ 点じ て 出合う を 折り曲げる 。 </S>
たった 先生 の 言葉 を 頼ん だ 。 </S>
さっさと 断り 誤っ た 私 に は 、 いか 彼 の 心持 を 付く よう に し た 。 </S>
私 の 愛し なかっ た 。 </S>
私 の 所 で 書物 が 宅 の いる その 後 なら 、 奥さん の 死ぬ の です 。 </S>
私 の 私 は そう な の 想像 を 起し た 。 </S>
お嬢さん の 叔父 で あっ た 。 </S>
私 の 方 から 出よ う 。 </S>
私 の 態度 も 時々 、 鈍い お嬢さん に は なかっ た 。 </S>
Ｋ から 見れ ば 構わ ない と 推定 し まし た 。 </S>
私 の 過去 に 我と を とうから 後らし て 新橋 を 鷹揚 すれ ば 霊 が ある 。 </S>
もっとも なぜ 世の中 で なく いっ て 。 </S>
私 の 強い し た の です 。 </S>
「 私 の 時 、 自分 の 気 に 聞い た 。 </S>
それ を 犬 は 大胆 だ 。 </S>
Ｋ の 経験 で あっ た 。 </S>
頼ま せる の で あっ た 。 </S>
私 は 今 の 眼 に 二度と 考え まし た 。 </S>
私 の

## 参考リンク

- 元論文
    - [SeqGAN: Sequence Generative Adversarial Nets with Policy Gradient](https://arxiv.org/abs/1609.05473)
- 作者実装(TensorFlow)
    - https://github.com/LantaoYu/SeqGAN
- 実装(PyTorch)
    - https://github.com/ZiJianZhao/SeqGAN-PyTorch


- 【強化学習に強くなりたい方向け】
    - An Introduction to Reinforcement Learning, Sutton and Barto, 1998
        - 強化学習の教科書（英語）、ネットでPDFが公開されてる
    - [Reinforcement Learning (DQN) tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#)
        - PyTorchによる強化学習（DQN）のTutorial


- 【意欲のある方向け】次に読むと面白いかもしれない論文リスト
    - [Adversarial Feature Matching for Text Generation](https://arxiv.org/abs/1706.03850) in ICML2017
        - SeqGANのupdate版
    - [Best of Both Worlds: Transferring Knowledge from Discriminative Learning to a Generative Visual Dialog Model](http://papers.nips.cc/paper/6635-best-of-both-worlds-transferring-knowledge-from-discriminative-learning-to-a-generative-visual-dialog-model) in NIPS2017
        - Visual Dialog Generation × GAN
    - [Improving Neural Machine Translation with Conditional Sequence Generative Adversarial Nets](https://arxiv.org/abs/1703.04887) in NAACL2018
        - 機械翻訳 × GAN